In [3]:
#    MARS SEASONS
#    ~~~~~~~~~~~~
#
#    - Mars mission data processor by season.
#    - Author: Raul Jesus Lopez @la9una
#    - GitHub: https://github.com/la9una/mars_habitability
#
#    This script processes environmental data from the Mars2020 mission,
#    adding a 'Season' column based on Martian solar longitude (L_s) 
#    and generating a representative sample for each season.
#    
#    Actions:
#    1. Download data from GitHub.
#    2. Add seasons based on L_s:
#       - Spring: 0° ≤ L_s < 90°
#       - Summer: 90° ≤ L_s < 180°
#       - Autumn: 180° ≤ L_s < 270°
#       - Winter: 270° ≤ L_s < 360°
#    3. Create a stratified sample:
#       - Select 30 samples or 10% of records per season (whichever is larger).
#       - Include all records if fewer than 30 are available.
#    4. Save:
#       - Full dataset with seasons.
#       - Representative sample.
#
#    Data Source:
#    - Time equivalence table from Perseverance (Mars 2020) Analyst's Notebook:
#      https://an.rsl.wustl.edu/m20/AN/an3.aspx
#
#    Output Files:
#    - generated_files/generated_mars2020_time_table_with_seasons.csv: Full dataset with seasons.
#    - generated_files/generated_mars2020_time_table_sample.csv: Representative sample.

import pandas as pd
import requests
import os
from io import StringIO
from IPython.display import display, FileLink

# Check if running in Google Colab
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    from google.colab import files
    print("Notebook running in Google Colab. Files will be downloadable.")
else:
    print("Notebook not running in Google Colab.")
    # Create the output folder if it doesn't exist (only for local runs)
    output_folder = "generated_files"
    os.makedirs(output_folder, exist_ok=True)

def download_csv(url):
    """Downloads a CSV file from a given URL."""
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(StringIO(response.text))

# Define the URL for the Mars data
url = "https://raw.githubusercontent.com/la9una/mars_habitability/refs/heads/main/mars2020_perseverance_time_table.csv"
data = download_csv(url)

def determine_season(solar_longitude):
    """Determines the Martian season based on the solar longitude (L_s)."""
    if 0 <= solar_longitude < 90:
        return 'Spring'
    elif 90 <= solar_longitude < 180:
        return 'Summer'
    elif 180 <= solar_longitude < 270:
        return 'Autumn'
    elif 270 <= solar_longitude < 360:
        return 'Winter'
    else:
        return 'Unknown'

def process_csv(data):
    """Processes the CSV and adds a 'Season' column."""
    data['Solar longitude (deg)'] = data['Solar longitude (deg)'].str.replace(',', '.').astype(float)
    data['Season'] = data['Solar longitude (deg)'].apply(determine_season)
    return data

# Process the data
data_with_seasons = process_csv(data)

def sample_by_season(data, min_sample_size=30):
    """Generates a representative sample by season."""
    samples = []
    for season in data['Season'].unique():
        season_data = data[data['Season'] == season]
        sample_size = max(int(0.1 * len(season_data)), min_sample_size)
        sample = season_data.sample(n=min(sample_size, len(season_data)), random_state=42, replace=False)
        samples.append(sample)
    return pd.concat(samples)

# Generate the sample
sample_data = sample_by_season(data_with_seasons)

# Define file paths (local or Colab)
if IN_COLAB:
    file_path_with_seasons = "generated_mars2020_time_table_with_seasons.csv"
    file_path_sample = "generated_mars2020_time_table_sample.csv"
else:
    file_path_with_seasons = os.path.join(output_folder, "generated_mars2020_time_table_with_seasons.csv")
    file_path_sample = os.path.join(output_folder, "generated_mars2020_time_table_sample.csv")

# Save the files
data_with_seasons.to_csv(file_path_with_seasons, index=False)
sample_data.to_csv(file_path_sample, index=False)

print(f'Processed data saved to: {file_path_with_seasons}')
print(f'Sampled data saved to: {file_path_sample}')

# Provide download links for Jupyter/GitHub environments
display(FileLink(file_path_with_seasons))
display(FileLink(file_path_sample))

# Download the files if running in Google Colab
if IN_COLAB:
    files.download(file_path_with_seasons)
    files.download(file_path_sample)


Processed data saved to: m20_perseverance_with_seasons.csv
Sampled data saved to: m20_perseverance_sample.csv
